DD/MM/YY = 06/02/01

ABC = 051

FIRST = PRINCY

LAST = GAUTAM

# Question 01

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision.transforms as transforms

In [2]:
# Define the target 5 classes
target_classes = [1, 3, 5, 7, 9]

In [3]:
# Define the custom dataset class
class MNIST_Dataset(Dataset):
    def __init__(self, data, target, transform=None):
        self.data = data
        self.target = target
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        image = self.data[idx].reshape(28, 28)
        label = self.target[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [4]:
# Load the MNIST dataset
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
X, y = mnist['data'], mnist['target']

In [5]:
# Convert the target to numpy arrays and cast to int type
y = y.astype(np.int64)

In [6]:
# Filter the data to include only the target 5 classes
X_filtered = X[np.isin(y, target_classes)]
y_filtered = y[np.isin(y, target_classes)]

In [7]:
# Split the data into training and test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=42)

In [8]:
# Convert the data to tensors
X_train = torch.tensor(X_train.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.int64)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_test = torch.tensor(y_test.values, dtype=torch.int64)


In [18]:
import random
class RandomGaussianNoise(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        noise = tensor.new().resize_as_(tensor).normal_(self.mean, self.std)
        return tensor + noise

data_transform = transforms.Compose([transforms.RandomHorizontalFlip(), RandomGaussianNoise(0, 0.05)])

In [10]:
# Create the custom datasets
train_dataset = MNIST_Dataset(X_train, y_train, transform=data_transform)
test_dataset = MNIST_Dataset(X_test, y_test)

In [11]:
# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=100, shuffle=False)

In [12]:
# Define the weight initialization function
def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.xavier_normal_(m.weight)

In [13]:
# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3, padding=1)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(in_features=7 * 7 * 32, out_features=512)
        #self.fc2 = nn.Linear(in_features=512, out_features=5)
        
    def forward(self, x):
        x = torch.unsqueeze(x, dim=1)
        x = self.conv1(x)
        x = torch.relu(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = torch.relu(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = torch.relu(x)
        x = x.view(-1, 7 * 7 * 32)
        x = self.fc1(x)
        x = torch.relu(x)
        #x = self.fc2(x)
        return x

In [14]:
# Initialize the model and set the optimizer
model = CNN()
model.apply(weights_init)

CNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool1): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool2): AvgPool2d(kernel_size=2, stride=2, padding=0)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=1568, out_features=512, bias=True)
)

In [15]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = nn.CrossEntropyLoss()

In [16]:
# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print('Epoch %d loss: %.3f' % (epoch + 1, running_loss / (i + 1)))
print('Finished training')

Epoch 1 loss: 0.357
Epoch 2 loss: 0.125
Epoch 3 loss: 0.091
Epoch 4 loss: 0.070
Epoch 5 loss: 0.060
Epoch 6 loss: 0.054
Epoch 7 loss: 0.048
Epoch 8 loss: 0.040
Epoch 9 loss: 0.041
Epoch 10 loss: 0.036
Finished training


In [17]:
# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

Accuracy of the network on the test images: 98 %
